<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Implementations/blob/master/Reinforcement%20Learning/Reinforcement%20Learning%20for%20Stock%20Market%20Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Author: Satwik Ram K

Reinforcement Learning

## Installing and Importing Dependencies

In [14]:
!pip install pandas-datareader

In [15]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import tensorflow as tf
from tensorflow import keras
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque
from tensorflow.keras import layers

## Building The AI Trader Network

In [61]:
class AI_Trader():

  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):

    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.model_name = model_name
    self.inventory = []

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()

  def model_builder(self):

    model = tf.keras.models.Sequential()

    model.add(layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))

    model.add(layers.Dense(units = 64, activation = "relu"))

    model.add(layers.Dense(units = 128, activation = "relu"))

    model.add(layers.Dense(units = self.action_space, activation = "linear"))

    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.001), loss = 'mse')

    return model

  def trade(self, state):

    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)

    return np.argmax(actions[0])


  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Data Preprocessing

### Sigmoid Function

In [62]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

### Price Format Function

In [63]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

### Dataset Loader

In [64]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

### State Creater

In [65]:
def state_creator(data, timestep, window_size): 

  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

## Loading the Dataset

In [66]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [67]:
data.head()

Date
2015-08-03    118.440002
2015-08-04    114.639999
2015-08-05    115.400002
2015-08-06    115.129997
2015-08-07    115.519997
Name: Close, dtype: float64

## Defining the model

In [68]:
window_size = 10
episodes = 20

batch_size = 32
data_samples = len(data) - 1


In [69]:
trader = AI_Trader(window_size)

In [70]:
trader.model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_17 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_18 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


## Training

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    


  0%|          | 0/1256 [00:00<?, ?it/s]

Episode: 1/20
AI Trader bought:  $ 118.440002
AI Trader sold:  $ 115.400002  Profit: - $ 3.040001
AI Trader bought:  $ 109.690002
AI Trader bought:  $ 112.919998
AI Trader bought:  $ 112.760002
AI Trader sold:  $ 107.720001  Profit: - $ 1.970001
AI Trader sold:  $ 112.339996  Profit: - $ 0.580002
AI Trader bought:  $ 109.269997
AI Trader bought:  $ 112.309998
AI Trader sold:  $ 110.150002  Profit: - $ 2.610001
AI Trader bought:  $ 114.209999
AI Trader sold:  $ 116.279999  Profit: $ 7.010002
AI Trader sold:  $ 116.410004  Profit: $ 4.100006



  3%|▎         | 33/1256 [00:02<01:47, 11.33it/s]

AI Trader sold:  $ 113.449997  Profit: - $ 0.760002



  3%|▎         | 34/1256 [00:05<18:48,  1.08it/s]

...

AI Trader bought:  $ 99.440002



 10%|▉         | 121/1256 [04:35<57:06,  3.02s/it]

AI Trader sold:  $ 99.989998  Profit: $ 0.549995



 10%|█         | 127/1256 [04:54<58:26,  3.11s/it]

AI Trader bought:  $ 96.349998
